## WebExtensions Disabled Intermittents Report

This notebook queries bugzilla for the WebExtension intermittents that have been disabled (and merge the fresh data coming from bugzilla with the additional data loaded from a spreasheet where some notes weere initially collected, e.g. the platforms on which as been disabled and if one of them is a core platform). 


In [17]:
import bugzilla
import pandas as pd
from datetime import datetime
import urllib.parse as urlparse

URL = "bugzilla.mozilla.org"
BZ_COMPONENTS = [
    "WebExtensions: Android",
    "WebExtensions: Compatibility",
    "WebExtensions: Developer Tools",
    "WebExtensions: Experiments",
    "WebExtensions: Frontend",
    "WebExtensions: General",
    "WebExtensions: Request Handling",
    "WebExtensions: Storage",
    "WebExtensions: Themes",
    "WebExtensions: Untriaged"
]

bzapi = bugzilla.Bugzilla(URL)

def format_see_also(see_also):
    out = []
    for link in see_also:
        if link.startswith("https://bugzilla.mozilla.org/show_bug.cgi?id="):
            bugid = urlparse.parse_qs(urlparse.urlparse(link).query)["id"][0]
            out += ["<a href='%s'>%s</a>" % (link, bugid)]
        else:
            out += ["<a href='%s'>%s</a>" % (link, link)]
    return ", ".join(out)
            

def report_webext_disabled_intermittents():
    ## Loads Bob's spreadsheet into a DataFrame and index it by bug number.
    csvDf = pd.read_csv("webext-disabled-tests.csv")
    csvDf["id"] = csvDf["Bug Number"]
    csvDf = csvDf.set_index("id")

    display('Number of notes related to disabled intermittents loaded from csv: %d' % len(list(csvDf.index)))
    
    query = bzapi.build_query(product="toolkit", component=BZ_COMPONENTS)
    query["keywords"] = "intermittent-failure"
    query["whiteboard"] = "disabled"
    query["is_open"] = True

    bugs = bzapi.query(query)

    display('Number of disabled intermittents found on bugzilla: %d' % len(bugs))

    ## Convert bugzilla query results into pandas DataFrame and prepare it to be joined
    ## with the DataFrame loaded from the csv file.
    df = pd.DataFrame.from_dict([bug.__dict__ for bug in bugs])
    
    # Set the index to the bug number (same as the DataFrame loaded from the csv file),
    # and join the two DataFrames.
    df = df.set_index("id")
    df = df.join(csvDf)
    
    # Duplicate the bug id column (to render it as an HTML link in the final rendered table)
    df["&#x1F517;"] = list(df.index);
    
    # Select the set of columns to render.
    df = df.loc[:, [
       "&#x1F517;",
       "Test", "summary",
       "priority",
       "Disabled on", "Core Platform",
       # "whiteboard", 
       "Assigned to", "assigned_to",
       "see_also",
       "last_change_time"
    ]]
    
    ## Use the summary where the Test column from the csv is empty.
    df["Test"] = df["Test"].fillna(df["summary"])
    df.drop('summary', axis=1, inplace=True)

    # Drop the csv "Assigned to" column in favor of "assigned_to" coming from the bugzilla query,
    # clean any nobody and remove the domain from the email addresses.
    df.drop('Assigned to', axis=1, inplace=True)
    df["assigned_to"] = df["assigned_to"].map(lambda x: "-" if x == "nobody@mozilla.org" else x.split("@")[0])

    ## Convert xmlrpc datetime object to string (to be able to sort by it).
    df["last_change_time"] = df["last_change_time"].map(lambda x: "%s" % x)

    ## Sort the table, fill any gap and format some columns for the final table rendering.
    table = df.sort_values(
        by=["priority", "Core Platform", "last_change_time"], ascending=[True, False, True]
    ).fillna("-").style.format({
        "&#x1F517;": lambda x: "<a href='https://bugzilla.mozilla.org/%s'>&#x1F517;</a>" % x,
        "see_also": format_see_also,
    })
    
    display("Last generated: %s" % datetime.today())
    
    display(table)

In [18]:
report_webext_disabled_intermittents()

'Number of notes related to disabled intermittents loaded from csv: 15'

'Number of disabled intermittents found on bugzilla: 18'

'Last generated: 2018-03-02 18:07:36.021260'

,🔗,Test,priority,Disabled on,Core Platform,assigned_to,see_also,last_change_time
id,,,,,,,,
1434590,🔗,browser_ext_tabs_lastAccessed.js,--,Windows,Yes,lgreco,,20180302T16:43:07
1434777,🔗,test_ext_contentscript_devtools_metadata.html,--,Windows,Yes,lgreco,,20180302T16:43:18
1417052,🔗,"Intermittent browser/components/extensions/test/browser/test-oop-extensions/browser_ext_omnibox.js | Expected suggestion to have title: ""select a"". - Got , expected select a",--,-,-,-,,20180301T05:49:31
1438663,🔗,Intermittent browser/components/extensions/test/browser/test-oop-extensions/browser_ext_popup_focus.js | Test timed out -,P1,-,-,mixedpuppy,,20180226T01:29:47
1315829,🔗,test_ext_runtime_onInstalled_and_onStartup.js,P2,All,Yes,-,,20171030T19:20:20
1258897,🔗,test_ext_sendmessage_reply2.html,P2,All,Yes,-,,20171030T19:24:38
1407501,🔗,test_ext_contentscript_scriptCreated.js,P2,All,Yes,kmaglione+bmo,,20180212T01:03:13
1285500,🔗,browser_ext_browserAction_popup.js,P2,"Linux, Windows",Yes,-,,20180226T01:08:07
1398120,🔗,test_ext_webrequest_responseBody.html,P2,Linux,No,kmaglione+bmo,,20180205T01:12:27
